In [1]:
import pandas as pd
import re

from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import wordnet

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
df = pd.read_csv('./Data/casual_political.csv')

In [3]:
df

,binary_sub,NN,NNP,VBG,PRP,VBP,TO,VB,IN,CC,...,RB,RBR,PRP$,VBN,MD,vader_neg,vader_pos,vader_neu,vader_compond,lemma_totaltext
0,1,0.079545,0.034091,0.011364,0.034091,0.045455,0.034091,0.079545,0.147727,0.034091,...,0.056818,0.011364,0.011364,0.011364,0.022727,0.042,0.103,0.854,0.6868,How important be the legislative achievement o...
1,0,0.233184,0.031390,0.013453,0.017937,0.026906,0.035874,0.044843,0.103139,0.044843,...,0.080717,0.000000,0.026906,0.008969,0.004484,0.016,0.109,0.874,0.9644,Does anybody use music song as their life s ti...
2,0,0.203704,0.018519,0.018519,0.055556,0.018519,0.018519,0.074074,0.092593,0.018519,...,0.074074,0.000000,0.055556,0.074074,0.037037,0.062,0.000,0.938,-0.5267,My school be stupid The teacher have be instru...
3,0,0.155844,0.051948,0.038961,0.103896,0.038961,0.012987,0.025974,0.129870,0.025974,...,0.051948,0.000000,0.051948,0.025974,0.000000,0.063,0.120,0.817,0.6996,Anyone wanna just chat about life I finish my ...
4,1,0.166667,0.083333,0.020833,0.020833,0.020833,0.020833,0.041667,0.083333,0.020833,...,0.083333,0.000000,0.000000,0.020833,0.000000,0.000,0.059,0.941,0.4696,How do Kentucky a heavily red state elect only...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,1,0.118421,0.105263,0.026316,0.026316,0.105263,0.013158,0.013158,0.065789,0.078947,...,0.052632,0.013158,0.000000,0.000000,0.000000,0.047,0.102,0.851,0.5803,Are there other news program that call out whe...
20054,0,0.129944,0.045198,0.045198,0.135593,0.033898,0.039548,0.045198,0.096045,0.045198,...,0.062147,0.000000,0.028249,0.033898,0.005650,0.083,0.070,0.847,-0.5520,I get scammed on Facebook and I m really hurt ...
20055,0,0.062500,0.062500,0.041667,0.104167,0.104167,0.041667,0.062500,0.104167,0.041667,...,0.083333,0.000000,0.000000,0.000000,0.000000,0.106,0.088,0.805,-0.2461,Why be it so hard on Christmas Eve Eve to find...
20056,0,0.140000,0.060000,0.000000,0.120000,0.060000,0.020000,0.000000,0.200000,0.020000,...,0.060000,0.000000,0.000000,0.000000,0.000000,0.000,0.150,0.850,0.7748,What s an album that you like every single son...


In [4]:
X = df.drop(columns='binary_sub')
y = df['binary_sub']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=16)

In [6]:
#X_train

In [7]:
tvec = TfidfVectorizer(stop_words='english', min_df=3)

In [8]:
X_train_tvec = tvec.fit_transform(X_train['lemma_totaltext'])

X_test_tvec = tvec.transform(X_test['lemma_totaltext'])

In [9]:
vocab = tvec.get_feature_names()

In [10]:
train_tvec_df = pd.DataFrame(X_train_tvec.todense(), columns=vocab)
test_tvec_df = pd.DataFrame(X_test_tvec.todense(), columns=vocab)

In [11]:
#train_tvec_df

In [12]:
X_train = X_train.drop(columns='lemma_totaltext')

In [13]:
X_train.reset_index(drop=True, inplace=True)

In [14]:
train_df = pd.concat([X_train, train_tvec_df], axis=1, ignore_index=True)

In [15]:
#train_df

In [16]:
X_test = X_test.drop(columns='lemma_totaltext')

In [17]:
X_test.reset_index(drop=True, inplace=True)

In [18]:
test_df = pd.concat([X_test, test_tvec_df], axis=1)

In [19]:
lr = LogisticRegression()

In [20]:
lr.fit(train_df, y_train)

LogisticRegression()

In [21]:
train_accuracy = lr.score(train_df, y_train)
train_accuracy

0.993456313099838

In [22]:
test_accuracy = lr.score(test_df, y_test)
test_accuracy

0.9850448654037887

In [23]:
# df['lemma_text'] = df['selftext'].apply(lambda x: lemmatize_text(x))

# df['lemma_title'] = df['title'].apply(lambda x: lemmatize_text(x))

# df['totaltext'] = df['lemma_title'] + df['lemma_text']

# tvec = TfidfVectorizer(stop_words='english', min_df=3)
# x = tvec.fit_transform(df['totaltext'])

# df_1 = pd.DataFrame(tvec.transform(df['totaltext']).todense(),
#                    columns=vocab)

# df_1 = df_1.T

# df_1['total_tfidf_score'] = df_1.sum(axis=1)

# df_1

